In [ ]:
%matplotlib inline

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pytesseract
import skimage.segmentation

# Feature detection

In [ ]:
NO_FEATURES = 2000
orb = cv2.ORB_create(NO_FEATURES)
matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)

## Reference image

In [ ]:
NO_DOTS = 40
NO_DOTS_MARGIN = 10
NO_DOTS_TOTAL = 9 * NO_DOTS + 2 * NO_DOTS_MARGIN + 6 * 1 + 4 * 2
LOCS_LINE = [
    NO_DOTS_MARGIN                   ,
    NO_DOTS_MARGIN               +  1,
    NO_DOTS_MARGIN + 1 * NO_DOTS +  2,
    NO_DOTS_MARGIN + 2 * NO_DOTS +  3,
    NO_DOTS_MARGIN + 3 * NO_DOTS +  4,
    NO_DOTS_MARGIN + 3 * NO_DOTS +  5,
    NO_DOTS_MARGIN + 4 * NO_DOTS +  6,
    NO_DOTS_MARGIN + 5 * NO_DOTS +  7,
    NO_DOTS_MARGIN + 6 * NO_DOTS +  8,
    NO_DOTS_MARGIN + 6 * NO_DOTS +  9,
    NO_DOTS_MARGIN + 7 * NO_DOTS + 10,
    NO_DOTS_MARGIN + 8 * NO_DOTS + 11,
    NO_DOTS_MARGIN + 9 * NO_DOTS + 12,
    NO_DOTS_MARGIN + 9 * NO_DOTS + 13,
]

In [ ]:
channel_100 = np.full((NO_DOTS_TOTAL, NO_DOTS_TOTAL), 255, dtype=np.uint8)
channel_100[LOCS_LINE, NO_DOTS_MARGIN:-NO_DOTS_MARGIN] = 0
channel_100[NO_DOTS_MARGIN:-NO_DOTS_MARGIN, LOCS_LINE] = 0

img_100 = cv2.merge((channel_100, channel_100, channel_100))

In [ ]:
fig_100 = plt.figure(100)
ax_100 = fig_100.gca()
ax_100.imshow(img_100, cmap="gray")

In [ ]:
keypoints_101, descriptors_101 = orb.detectAndCompute(img_100, None)

In [ ]:
img_101 = cv2.drawKeypoints(
    img_100,
    keypoints_101,
    outImage = np.array([]),
    color = (255, 0, 0),
    flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
)

In [ ]:
fig_101 = plt.figure(101)
ax_101 = fig_101.gca()
ax_101.imshow(img_101)

## First screenshot

In [ ]:
img_200 = cv2.imread(
    "Screenshot from 2022-01-02 21-00-15.png",
    cv2.IMREAD_GRAYSCALE,
)

In [ ]:
fig_200 = plt.figure(200)
ax_200 = fig_200.gca()
ax_200 = ax_200.imshow(img_200, cmap="gray")

In [ ]:
keypoints_201, descriptors_201 = orb.detectAndCompute(img_200, None)

In [ ]:
img_201 = cv2.drawKeypoints(
    img_200,
    keypoints_201,
    outImage = np.array([]),
    color = (255, 0, 0),
    flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
)

In [ ]:
fig_201 = plt.figure(201)
ax_201 = fig_201.gca()
ax_201.imshow(img_201)

In [ ]:
matches_202 = list(matcher.match(descriptors_101, descriptors_201, None))
matches_202.sort(key=lambda x: x.distance, reverse=False
                )
matches_202 = matches_202[:len(matches_202) // 10]

In [ ]:
img_202 = cv2.drawMatches(img_100, keypoints_101, img_200, keypoints_201, matches_202, None)

In [ ]:
fig_202 = plt.figure(202)
ax_202 = fig_202.gca()
ax_202.imshow(img_202)

# Contour detection

In [ ]:
img_1000 = cv2.imread(
    "../assets/Screenshot from 2022-01-02 21-00-15.png",
    cv2.IMREAD_GRAYSCALE,
)

In [ ]:
fig_1000 = plt.figure(1000)
ax_1000 = fig_1000.gca()
ax_1000 = ax_1000.imshow(img_1000, cmap="gray")

In [ ]:
img_1001 = cv2.adaptiveThreshold(
    img_1000,
    255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV,
    11,
    2,
)

In [ ]:
fig_1001 = plt.figure(1001)
ax_1001 = fig_1001.gca()
ax_1001.imshow(img_1001, cmap="gray")

In [ ]:
cnts, hierarchy = cv2.findContours(img_1001, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnt = cnts[0]
print(cnt)

In [ ]:
M_1002 = cv2.getPerspectiveTransform(
    np.array([e[0] for e in cnt], dtype=np.float32),
    np.array([[0, 0], [0, 450], [450, 450], [450, 0]], dtype=np.float32),
)
img_1002 = cv2.warpPerspective(img_1001, M_1002, (450, 450))

In [ ]:
fig_1002 = plt.figure(1002)
ax_1002 = fig_1002.gca()
ax_1002.imshow(img_1002, cmap="gray")

In [ ]:
res = [[None for _ in range(9)] for _ in range(9)]

for i in range(9):
    x_left = i * 50
    x_right = (i + 1) * 50
    
    for j in range(9):
        y_left = j * 50
        y_right = (j + 1) * 50
        
        img_it = img_1002[x_left:x_right, y_left:y_right]
        img_it_wo_border = skimage.segmentation.clear_border(img_it, 3)
        
        res_it = pytesseract.image_to_string(
            cv2.bitwise_not(img_it_wo_border),
            config = "--psm 10 outputbase digits",
        )
        res_it = res_it.strip()
        
        if not res_it:
            continue
            
        res[i][j] = int(res_it)
        
        fig_it = plt.figure(1100 + 100 * i + 10 * j)
        ax_1, ax_2 = fig_it.subplots(1, 2)
        ax_1.imshow(img_it, cmap="gray")
        ax_2.imshow(img_it_wo_border, cmap="gray")

In [ ]:
print("\n".join([" ".join([str(c) if c is not None else " " for c in res_it]) for res_it in res]))